# RIN interactions

## Summary

Protein structures, dynamics and functions are the result of a network of interactions between the residues that constitute them. We call such network *Residue-Interaction Network* (RIN). 

In this notebook, we obtained the subset of the Nitrogenase RIN that connects the D and the G subunits by using the PROLIF library on extant and ancient structures that we predicted through ColabFold. Our aim is to construct the timeline of the evolution of this interface across time — though mostly focused on the emergence of the interface—



In [41]:
import prolif
import MDAnalysis as mda
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from typing import List, Dict
from pyfamsa import Aligner, Sequence
# import logomaker
import pandas as pd
import numpy as np
from scipy.stats import entropy
import prody as pdy
import matplotlib.pyplot as plt
from Bio import AlignIO
import prody as pdy
from Bio.SeqUtils import seq1
import seaborn as sns
from Bio import AlignIO


## Functions

In [42]:
def align_structure_sequences(proteins: Dict[str, pdy.AtomGroup], chain):
    # TODO: Move this code to a module
    """
    Generates a list of aligned sequences

    arguments
    ---------
    - proteins: List[pdy.AtomGroup]
    - chain: str
    
    """
    # Align Sequence
    sequence_records = []
    for key, protein in proteins.items():
        sequence_records.append(
            Sequence(key.encode(), protein.select(f'chain {chain:s} and name CA').getSequence().encode())
        )
    
    aligner = Aligner()
    msa = aligner.align(sequence_records)
    # return [dict(id=item.id.decode(), sequence=item.sequence.decode()) for item in msa]
    return [[item.id.decode()] + list(item.sequence.decode()) for item in msa]


def map_alignment_to_structure(alignment, structures, on_chain):
    """
    Returns a dataframe with the sequence alignment and mappings to its residues

    arguments
    ---------
    - proteins: List[pdy.AtomGroup]
    - chain: str

    """
    alignment = pd.DataFrame(alignment).set_index(0).T
    for column in alignment.columns:
        structure = structures[column]
        seq = alignment[column].to_list()
        chain = structure.select(f'chain {on_chain:s}').getHierView()
        residues = list(chain.iterResidues())
        residue_list = []
        active = False
        for i, pos in enumerate(seq):
            if len(residues) == 0:
                residue_list.append(None)
                continue
            
            if pos == '-': 
                residue_list.append(None)
                continue
            
            if pos == residues[0].select('name CA').getSequence():
                residue_list.append(residues.pop(0))

        if len(residues) > 0:
            raise RuntimeError("sequence failed to map")
        alignment['map:' + column] = residue_list

    return alignment

def get_position_entropy(u):
    """
    Useful to compute the entropy of the alignment
    """
    u = np.array(u)
    probs = []
    for char in np.unique(u):
        probs.append(((u == char).sum() / len(u)))
    
    return entropy(probs, base=2)


def get_coord_sets(alignment):
    coordinates = []
    for column in filter(lambda x: x[:3] == 'map', alignment.columns):
        coordinates.append(alignment.query('entropy < 0.01').apply(lambda x: x[column].select('name CA').getCoords().reshape(-1).tolist(), axis=1).to_list())
    return np.array(coordinates)

In [43]:
def generate_network(prmtop, pdb, reference, chain_D, chain_G, label1, label2):
    u = mda.Universe(prmtop, pdb)
    ref = mda.Universe(reference)
    u.add_TopologyAttr('chainID')

    for res, res_ref in zip(u.residues, ref.residues):
        res.resnum = res_ref.resnum
        res.resid = res_ref.resid
        for atom in res.atoms:
            atom.chainID = np.unique(res_ref.atoms.chainIDs)[0]

    chain_D = u.select_atoms(chain_D)
    chain_G = u.select_atoms(chain_G)
    G = nx.DiGraph()
    
    pf = prolif.Fingerprint()
    pf.run(u.trajectory, chain_D, chain_G)

    for record in pf.to_dataframe().T.to_records():
        label_1 = label1 + '.' + record[0].replace('HIE', 'HIS')
        chain_1 = record[0].split('.')[1]
        residue_1 = record[0].split('.')[0].replace('HIE', 'HIS')
        positions_1 = u.select_atoms(f'chainid {chain_1} and resnum {residue_1[3:]} and name CA').positions

        label_2 = label2 + '.' + record[1].replace('HIE', 'HIS')
        chain_2 = record[1].split('.')[1]
        residue_2 = record[1].split('.')[0].replace('HIE', 'HIS')
        positions_2 = u.select_atoms(f'chainid {chain_2} and resnum {residue_2[3:]} and name CA').positions


        G.add_node(label_1, chain=chain_1, residue=residue_1, positions=positions_1)
        G.add_node(label_2, chain=chain_2, residue=residue_2, positions=positions_2)
        G.add_edge(label_1, label_2, type=record[2])

    return G, pf

restype = dict(
    ALA = "apolar",
    CYS = "polar",
    ASP = "negative",
    GLU = "negative",
    PHE = "apolar",
    GLY = "polar",
    HIS = "positive",
    HIE = "positive",
    ILE = "apolar",
    LYS = "positive",
    LEU = "apolar",
    MET = "apolar",
    ASN = "very_polar",
    PRO = "apolar",
    GLN = "very_polar",
    ARG = "positive",
    SER = "polar",
    THR = "polar",
    VAL = "apolar",
    TRP = "bulky",
    TYR = "polar"
)

## Evolution of the interface

### Sequence Alignment

One of the main issues we face is that residues change their numbering during evolution due to insertions and deletions. We map all the proteins that we use to a common dataframe after aligning their sequences, so we skip this problem. **This data will be used in other analysis!**.

#### D subunit

In [44]:
files = [
    'DK.Anc265', 'DK.Anc264', 'DKG.Anc262', 'DKG.Anc263','DKG.Anc330', 'DKG.AzoV.Anf.pdb','DKG.AzoV.Vnf'
]
structures = dict((file, pdy.parsePDB(f'../data/surface-network/{file:s}.aligned.pdb')) for file in files)
alignment = align_structure_sequences(structures, chain='A')
u = map_alignment_to_structure(alignment, structures, 'A')
seq_columns = [col for col in u.columns if col[:3] != 'map']
u['entropy'] = u[seq_columns].apply(lambda x: get_position_entropy(x), axis=1)
# alignment_coordinates = get_coord_sets(u)

#### G subunit

In [45]:
files = [
    'DKG.Anc262', 'DKG.Anc263','DKG.Anc330', 'DKG.AzoV.Anf.pdb','DKG.AzoV.Vnf'
]
structures = dict((file, pdy.parsePDB(f'../data/surface-network/{file:s}.aligned.pdb')) for file in files)
alignment = align_structure_sequences(structures, chain='C')
v = map_alignment_to_structure(alignment, structures, 'C')
seq_columns = [col for col in v.columns if col[:3] != 'map']
v['entropy'] = v[seq_columns].apply(lambda x: get_position_entropy(x), axis=1)

In [46]:
# Mapping Residues D subunit
u['resnum:AzoV.Vnf'] = u['map:DKG.AzoV.Vnf'].apply(lambda x: int(x.select('name CA').getResnums()[0]) if x is not None else None).astype(int, errors='ignore')
u['resnum:AzoV.Anf'] = u['map:DKG.AzoV.Anf.pdb'].apply(lambda x: int(x.select('name CA').getResnums()[0]) if x is not None else None).astype(int, errors='ignore')
u['resnum:Anc262'] = u['map:DKG.Anc262'].apply(lambda x: int(x.select('name CA').getResnums()[0]) if x is not None else None).astype(int, errors='ignore')
u['resnum:Anc263'] = u['map:DKG.Anc263'].apply(lambda x: int(x.select('name CA').getResnums()[0]) if x is not None else None).astype(int, errors='ignore')
u['resnum:Anc330'] = u['map:DKG.Anc330'].apply(lambda x: int(x.select('name CA').getResnums()[0]) if x is not None else None).astype(int, errors='ignore')
u['resnum:Anc265'] = u['map:DK.Anc265'].apply(lambda x: int(x.select('name CA').getResnums()[0]) if x is not None else None).astype(int, errors='ignore')
u['resnum:Anc264'] = u['map:DK.Anc264'].apply(lambda x: int(x.select('name CA').getResnums()[0]) if x is not None else None).astype(int, errors='ignore')
# Mapping Residues G subunit
v['resnum:AzoV.Vnf'] = v['map:DKG.AzoV.Vnf'].apply(lambda x: int(x.select('name CA').getResnums()[0]) if x is not None else None).astype(int, errors='ignore')
v['resnum:AzoV.Anf'] = v['map:DKG.AzoV.Anf.pdb'].apply(lambda x: int(x.select('name CA').getResnums()[0]) if x is not None else None).astype(int, errors='ignore')
v['resnum:Anc262'] = v['map:DKG.Anc262'].apply(lambda x: int(x.select('name CA').getResnums()[0]) if x is not None else None).astype(int, errors='ignore')
v['resnum:Anc263'] = v['map:DKG.Anc263'].apply(lambda x: int(x.select('name CA').getResnums()[0]) if x is not None else None).astype(int, errors='ignore')
v['resnum:Anc330'] = v['map:DKG.Anc330'].apply(lambda x: int(x.select('name CA').getResnums()[0]) if x is not None else None).astype(int, errors='ignore')

In [47]:
signature_values_u = pd.read_csv('../data/signatures/signatures.csv').query('chain == "A"').rename(columns={"residue": "resnum:AzoV.Vnf"})
signature_values_v = pd.read_csv('../data/signatures/signatures.csv').query('chain == "C"').rename(columns={"residue": "resnum:AzoV.Vnf"})

In [48]:
u = pd.merge(u, signature_values_u, on='resnum:AzoV.Vnf', how='inner')
v = pd.merge(v, signature_values_v, on='resnum:AzoV.Vnf', how='inner')

### Networks

In [49]:
anc262_files = (
    '../data/surface-network/DKG.Anc262.aligned.prmtop', 
    '../data/surface-network/DKG.Anc262.aligned.inpcrd', 
    '../data/surface-network/DKG.Anc262.aligned.pdb', 'chainid A', 'chainid C', 'Anc262', 'Anc262'
)

anc263_files = (
    '../data/surface-network/DKG.Anc263.aligned.prmtop', 
    '../data/surface-network/DKG.Anc263.aligned.inpcrd', 
    '../data/surface-network/DKG.Anc263.aligned.pdb', 'chainid A', 'chainid C', 'Anc263', 'Anc263'
)

anc330_files = (
    '../data/surface-network/DKG.Anc330.aligned.prmtop', 
    '../data/surface-network/DKG.Anc330.aligned.inpcrd', 
    '../data/surface-network/DKG.Anc330.aligned.pdb', 'chainid A', 'chainid C', 'Anc330', 'Anc330'
)
azov_vnf = (
    '../data/surface-network/DKG.AzoV.Vnf.aligned.prmtop', 
    '../data/surface-network/DKG.AzoV.Vnf.aligned.inpcrd', 
    '../data/surface-network/DKG.AzoV.Vnf.aligned.pdb', 'chainid A', 'chainid C', 'AzoV.Vnf', 'AzoV.Vnf'
)

azov_anf = (
    '../data/surface-network/DKG.AzoV.Anf.aligned.prmtop', 
    '../data/surface-network/DKG.AzoV.Anf.aligned.inpcrd', 
    '../data/surface-network/DKG.AzoV.Anf.pdb.aligned.pdb', 'chainid A', 'chainid C', 'AzoV.Anf', 'AzoV.Anf'
)

In [50]:
anc262_network_original, anc262_df = generate_network(*anc262_files)
anc263_network_original, anc263_df = generate_network(*anc263_files)
anc330_network_original, anc330_df = generate_network(*anc330_files)
azov_vnf_network_original, azov_vnf_df = generate_network(*azov_vnf)
azov_anf_network_original, azov_anf_df = generate_network(*azov_anf)

  0%|          | 0/1 [00:00<?, ?it/s]/home/bcz/miniconda3/envs/gsubunit/lib/python3.9/multiprocessing/reduction.py:51: UserWarning: Reader has no dt information, set to 1.0 ps
  cls(buf, protocol).dump(obj)
/home/bcz/miniconda3/envs/gsubunit/lib/python3.9/site-packages/MDAnalysis/converters/RDKit.py:990: UserWarning: The standardization could not be completed within a reasonable number of iterations
  warnings.warn("The standardization could not be completed within a "
  0%|          | 0/1 [00:00<?, ?it/s]/home/bcz/miniconda3/envs/gsubunit/lib/python3.9/multiprocessing/reduction.py:51: UserWarning: Reader has no dt information, set to 1.0 ps
  cls(buf, protocol).dump(obj)
/home/bcz/miniconda3/envs/gsubunit/lib/python3.9/site-packages/MDAnalysis/converters/RDKit.py:990: UserWarning: The standardization could not be completed within a reasonable number of iterations
  warnings.warn("The standardization could not be completed within a "
  0%|          | 0/1 [00:00<?, ?it/s]/home/bcz/minic

In [51]:
anc262_network = anc262_network_original.copy()
anc263_network = anc263_network_original.copy()
anc330_network = anc330_network_original.copy()
azov_vnf_network = azov_vnf_network_original.copy()
azov_anf_network = azov_anf_network_original.copy()

In [52]:
azov_vnf_d_residues = [int(item[0].split('.')[0][3:]) for item in azov_vnf_df.to_dataframe().T.index]
azov_anf_d_residues = [int(item[0].split('.')[0][3:]) for item in azov_anf_df.to_dataframe().T.index]
anc262_d_residues = [int(item[0].split('.')[0][3:]) for item in anc262_df.to_dataframe().T.index]
anc263_d_residues = [int(item[0].split('.')[0][3:]) for item in anc263_df.to_dataframe().T.index]
anc330_d_residues = [int(item[0].split('.')[0][3:]) for item in anc330_df.to_dataframe().T.index]

In [53]:
azov_vnf_g_residues = [int(item[1].split('.')[0][3:]) for item in azov_vnf_df.to_dataframe().T.index]
azov_anf_g_residues = [int(item[1].split('.')[0][3:]) for item in azov_anf_df.to_dataframe().T.index]
anc262_g_residues = [int(item[1].split('.')[0][3:]) for item in anc262_df.to_dataframe().T.index]
anc263_g_residues = [int(item[1].split('.')[0][3:]) for item in anc263_df.to_dataframe().T.index]
anc330_g_residues = [int(item[1].split('.')[0][3:]) for item in anc330_df.to_dataframe().T.index]

100%|██████████| 1/1 [00:34<00:00, 34.14s/it]

### Principal Component Decomposition

RINs will be visualized with tools for network analysis. Those usually consider 2D planes to visualize data. Given that we want to compare the networks among them, have a certain spatial feeling and represent as much data as possible, applying a Principal Component Analysis decomposition in the 3D coordinates of all the interface residues is the easiest path to comply with our three goals. We use the PCs of one of our systems and we apply it to all our other systems. *Note that our structures were previously aligned*.

In [54]:
ref_positions = []
for node, node_data in azov_vnf_network.nodes(data=True):
    try:
        ref_positions.append(node_data['positions'])
    except KeyError:
        print(f"problem with {node}")


ref_positions = np.concatenate(ref_positions, axis=0)
ref_positions = StandardScaler().fit_transform(ref_positions)
pca = PCA(3).fit(ref_positions)


Transforming coordinates.

In [55]:
# 
for network in [anc263_network, anc262_network, anc330_network, azov_vnf_network, azov_anf_network]:
    positions = []
    for node, node_data in network.nodes(data=True):
        positions.append(node_data['positions'])
    positions = np.concatenate(positions, axis=0)
    positions = StandardScaler().fit_transform(positions)
    positions = pca.transform(positions)

    for (node, node_data), position in zip(network.nodes(data=True), positions):
        node_data['pos_x'] = float(position[0])
        node_data['pos_y'] = float(position[1])
        node_data['residue_label'] = node_data['residue'][:3]
        del node_data['positions']        


In [56]:
for network in [anc263_network, anc262_network, anc330_network, azov_vnf_network, azov_anf_network]:
    for node, node_data in network.nodes(data=True):
        node_data['residue_type'] = restype[node_data['residue_label']]

### Mapping Networks to the sequence alignment

By doing this, we will be able to have a global picture of which residues are at the interface in each of the systems, and we will be able to generate a global interface list — even when there is not G subunit.

In [57]:
u['interface'] = False
u.loc[u['resnum:AzoV.Vnf'].isin(pd.unique(azov_vnf_d_residues)),'interface'] = True
u.loc[u['resnum:AzoV.Anf'].isin(pd.unique(azov_anf_d_residues)),'interface'] = True
u.loc[u['resnum:Anc262'].isin(pd.unique(anc262_d_residues)),'interface'] = True
u.loc[u['resnum:Anc263'].isin(pd.unique(anc263_d_residues)),'interface'] = True
u.loc[u['resnum:Anc330'].isin(pd.unique(anc330_d_residues)),'interface'] = True
u.query('interface == True').shape
#alignment = AlignIO.read(open("../data/surface-network/D_Align.fasta"), "fasta")

v['interface'] = False
v.loc[v['resnum:AzoV.Vnf'].isin(pd.unique(azov_vnf_g_residues)),'interface'] = True
v.loc[v['resnum:AzoV.Anf'].isin(pd.unique(azov_anf_g_residues)),'interface'] = True
v.loc[v['resnum:Anc262'].isin(pd.unique(anc262_g_residues)),'interface'] = True
v.loc[v['resnum:Anc263'].isin(pd.unique(anc263_g_residues)),'interface'] = True
v.loc[v['resnum:Anc330'].isin(pd.unique(anc330_g_residues)),'interface'] = True
v.query('interface == True').shape
#alignment = AlignIO.read(open("../data/surface-network/D_Align.fasta"), "fasta")

(33, 20)

In [58]:
def generate_index(series, label, chain):
    out = []
    for item in series:
        if item is None:
            out.append(None)
        else:
            resnum = item.getResnum()
            restype = item.getResname()
            out.append(label + '.{:s}{:d}'.format(restype, resnum) + '.' + chain)
    return out

In [59]:
u['index:Anc265'] = generate_index(u['map:DK.Anc265'], 'Anc265', 'A')
u['index:Anc264'] = generate_index(u['map:DK.Anc264'], 'Anc264', 'A')
u['index:Anc263'] = generate_index(u['map:DKG.Anc263'], 'Anc263', 'A')
u['index:Anc262'] = generate_index(u['map:DKG.Anc262'], 'Anc262', 'A')
u['index:Anc330'] = generate_index(u['map:DKG.Anc330'], 'Anc330', 'A')
u['index:AzoV.Anf'] = generate_index(u['map:DKG.AzoV.Anf.pdb'], 'AzoV.Anf', 'A')
u['index:AzoV.Vnf'] = generate_index(u['map:DKG.AzoV.Vnf'], 'AzoV.Vnf', 'A')

v['index:Anc263'] = generate_index(v['map:DKG.Anc263'], 'Anc263', 'C')
v['index:Anc262'] = generate_index(v['map:DKG.Anc262'], 'Anc262', 'C')
v['index:Anc330'] = generate_index(v['map:DKG.Anc330'], 'Anc330', 'C')
v['index:AzoV.Anf'] = generate_index(v['map:DKG.AzoV.Anf.pdb'], 'AzoV.Anf', 'C')
v['index:AzoV.Vnf'] = generate_index(v['map:DKG.AzoV.Vnf'], 'AzoV.Vnf', 'C')

We save here the D and G aligned residue tables.

In [60]:
u.to_csv('../data/surface-network/D.subunit.csv')
v.to_csv('../data/surface-network/G.subunit.csv')

### Including non-interacting residues in RINs

This step is aimed to make all networks uniform. Some of our systems, as Anc264 don't even have a small subunit, so the interface doesn't exist. However, we might still be interested in comparing the regions of the protein surface that could have participated in the interaction.


In [61]:
anc264_ni_network = nx.DiGraph()
for index, item in u.query('interface == True')['map:DK.Anc264'].items():
    if item is None: continue
    label = 'Anc264' + '.' + item.getResname() + '{:d}'.format(item.getResnum()) + '.A'
    anc264_ni_network.add_node(
        label, chain='A', residue=item.getResname().replace('HIE', 'HIS'), positions=item.select('name CA').getCoords()
    )


anc265_ni_network = nx.DiGraph()
for index, item in u.query('interface == True')['map:DK.Anc265'].items():
    if item is None: continue
    label = 'Anc265' + '.' + item.getResname() + '{:d}'.format(item.getResnum()) + '.A'
    anc265_ni_network.add_node(
        label, chain='A', residue=item.getResname().replace('HIE', 'HIS'), positions=item.select('name CA').getCoords()
    )

anc263_ni_network = nx.DiGraph()
for index, item in u.query('interface == True')['map:DKG.Anc263'].items():
    if item is None: continue
    label = 'Anc263' + '.' + item.getResname() + '{:d}'.format(item.getResnum()) + '.A'
    anc263_ni_network.add_node(
        label, chain='A', residue=item.getResname().replace('HIE', 'HIS'), positions=item.select('name CA').getCoords()
    )

for index, item in v.query('interface == True')['map:DKG.Anc263'].items():
    if item is None: continue
    label = 'Anc263' + '.' + item.getResname() + '{:d}'.format(item.getResnum()) + '.C'
    anc263_ni_network.add_node(
        label, chain='C', residue=item.getResname().replace('HIE', 'HIS'), positions=item.select('name CA').getCoords()
    )


anc262_ni_network = nx.DiGraph()
for index, item in u.query('interface == True')['map:DKG.Anc262'].items():
    if item is None: continue
    label = 'Anc262' + '.' + item.getResname() + '{:d}'.format(item.getResnum()) + '.A'
    anc262_ni_network.add_node(
        label, chain='A', residue=item.getResname().replace('HIE', 'HIS'), positions=item.select('name CA').getCoords()
    )

anc330_ni_network = nx.DiGraph()
for index, item in u.query('interface == True')['map:DKG.Anc330'].items():
    if item is None: continue
    label = 'Anc330' + '.' + item.getResname() + '{:d}'.format(item.getResnum()) + '.A'
    anc330_ni_network.add_node(
        label, chain='A', residue=item.getResname().replace('HIE', 'HIS'), positions=item.select('name CA').getCoords()
    )

azov_vnf_ni_network = nx.DiGraph()
for index, item in u.query('interface == True')['map:DKG.AzoV.Vnf'].items():
    if item is None: continue
    label = 'AzoV.Vnf' + '.' + item.getResname().replace('HIE', 'HIS') + '{:d}'.format(item.getResnum()) + '.A'
    azov_vnf_ni_network.add_node(
        label, chain='A', residue=item.getResname().replace('HIE', 'HIS'), positions=item.select('name CA').getCoords()
    )

azov_anf_ni_network = nx.DiGraph()
for index, item in u.query('interface == True')['map:DKG.AzoV.Anf.pdb'].items():
    if item is None: continue
    label = 'AzoV.Anf' + '.' + item.getResname().replace('HIE', 'HIS') + '{:d}'.format(item.getResnum()) + '.A'
    azov_anf_ni_network.add_node(
        label, chain='A', residue=item.getResname(), positions=item.select('name CA').getCoords()
    )


In [62]:
# 
for network in [anc265_ni_network, anc264_ni_network,anc263_ni_network,anc262_ni_network,anc330_ni_network,azov_vnf_ni_network,azov_anf_ni_network]:
    positions = []
    for node, node_data in network.nodes(data=True):
        positions.append(node_data['positions'])
    positions = np.concatenate(positions, axis=0)
    positions = StandardScaler().fit_transform(positions)
    positions = pca.transform(positions)

    for (node, node_data), position in zip(network.nodes(data=True), positions):
        node_data['pos_x'] = float(position[0])
        node_data['pos_y'] = float(position[1])
        node_data['residue'] = node.split('.')[-2]
        node_data['residue_label'] = node_data['residue'][:3]
        node_data['residue_type'] = restype[node_data['residue_label']]
        del node_data['positions']        


Composing the final networks

In [63]:
anc265_network = anc265_ni_network.copy()
anc264_network = anc264_ni_network.copy()
anc263_network = nx.compose_all([anc263_ni_network, anc263_network])
anc262_network = nx.compose_all([anc262_ni_network, anc262_network])
anc330_network = nx.compose_all([anc330_ni_network, anc330_network])
azov_vnf_network = nx.compose_all([azov_vnf_ni_network, azov_vnf_network])
azov_anf_network = nx.compose_all([azov_anf_ni_network, azov_anf_network])

In [64]:
#u['resnum:AzoV.Vnf'] = u['map:DKG.AzoV.Vnf'].apply(lambda x: x.getResnum())
#u['resnum:AzoV.Vnf'] = u['map:DKG.AzoV.Vnf'].apply(lambda x: x.getResnum())

We assign some more information that will be useful for our visualizations.

In [65]:
for network, column in [
    (anc265_network, 'index:Anc265'), (anc264_network, 'index:Anc264'),
    (anc263_network, 'index:Anc263'), (anc262_network, 'index:Anc262'), (anc330_network, 'index:Anc330'), 
    (azov_vnf_network, 'index:AzoV.Vnf'), (azov_anf_network, 'index:AzoV.Anf')]:
    for node, node_data in network.nodes(data=True):
        if node_data['chain'] == "C": 
            aln_ref = v
        else:
            aln_ref = u

        try:
            item = aln_ref.reset_index().set_index(column).loc[node]
        except KeyError:
            item = aln_ref.reset_index().set_index(column).loc[node.replace('HIE', 'HIS')]
        node_data['aln_position'] = int(item['resnum:AzoV.Vnf'])
        node_data['res'] = seq1(node_data['residue'][:3].replace('HIE', 'HIS'))

        node_data['cool_name'] = node_data['res'] + '{:d}'.format(node_data['aln_position'])

100%|██████████| 1/1 [00:35<00:00, 35.14s/it]


In [66]:
nx.write_gml(anc265_network, '../data/surface-network/anc265.gml')
nx.write_gml(anc264_network, '../data/surface-network/anc264.gml')
nx.write_gml(anc263_network, '../data/surface-network/anc263.gml')
nx.write_gml(anc262_network, '../data/surface-network/anc262.gml')
nx.write_gml(anc330_network, '../data/surface-network/anc330.gml')
nx.write_gml(azov_vnf_network, '../data/surface-network/AzoV.Vnf.gml')
nx.write_gml(azov_anf_network, '../data/surface-network/AzoV.Anf.gml')

In [67]:
def add_level(network, level):

    for node, node_data in network.nodes(data=True):
        node_data['level'] = level
    return network

Networks with our networks stacked according to their evolutionary timeline.

In [68]:
anc264_network = add_level(anc264_network, level=0)
anc263_network = add_level(anc263_network, level=1)
anc262_network = add_level(anc262_network, level=2)
azov_vnf_network = add_level(azov_vnf_network, level=3)
stacked = nx.compose_all([anc264_network, anc263_network, anc262_network, azov_vnf_network])
nx.write_gml(stacked, '../data/surface-network/stacked.vnf.gml')


In [69]:
anc263_network = add_level(anc263_network, level=1)
anc262_network = add_level(anc330_network, level=2)
azov_vnf_network = add_level(azov_anf_network, level=3)
stacked = nx.compose_all([anc263_network, anc330_network, azov_anf_network])
nx.write_gml(stacked, '../data/surface-network/stacked.anf.gml')

In [70]:
seq_interface = u.query('interface == True')[['DK.Anc265','DK.Anc264','DKG.Anc263','DKG.Anc262','DKG.Anc330','DKG.AzoV.Vnf', 'DKG.AzoV.Anf.pdb']].values.T.tolist()
seq_interface = '\n'.join([''.join(i) for i in seq_interface])

Showing interface alignments. These sequences don't represent the sequences but the residues that make those interfaces, devoid of any order.

In [71]:
print(seq_interface)

SENLCNHLRTYQMKEGPYIRSEQKREKFEWHK-HEDEKAR
DTNLCNHLRTYQMKEGPYIKDEYEREKFEWHK-HEDEKAR
DTTLSNHLRGYNEKKGPRLDDDYERKAFEWHKDHQEEKAR
DTTLSNHLRGYNEKKGPRLDDDYERKAFEWHKDHQEEKAR
DLTLGYHLREYNERKGPRLDDDLEMKMFEWHHEHQGEKAR
DTRLANHLRGYNEKKGPRLDDNYERKAFEWHKDHEEEKAR
TLALGYHLREYNERVGPRLDDKPDRKMFEWHHEHQGEKAR


In [72]:
u.query('interface == True').index

Int64Index([ 25,  26,  27,  30,  33,  34, 179, 250, 257, 260, 261, 264, 265,
            267, 268, 271, 273, 274, 275, 276, 278, 283, 284, 287, 290, 291,
            294, 295, 299, 339, 340, 343, 347, 362, 363, 364, 367, 368, 371,
            372],
           dtype='int64')

In [73]:
seq_interface = v.query('interface == True')[['DKG.Anc263','DKG.Anc262','DKG.Anc330','DKG.AzoV.Vnf', 'DKG.AzoV.Anf.pdb']].values.T.tolist()
seq_interface = '\n'.join([''.join(i) for i in seq_interface])

In [74]:
print(seq_interface)

QERLFYSDRELPQRCYADKIADKRDIKSLNEEY
QERLFYSDREMPQRCYADKIADKRDIKSLNEEY
MKRLFHSDRRLPARCWVDVVNGRRYLKSLNQEY
EERLFFSDRERPQRLYADLANDERDVRSTNREY
MKNLFHSDRRLSHRCWVDVCDDERYLGSLNEEY


In [75]:
v.query('interface == True')['resnum:AzoV.Vnf'].astype(int).values

array([ 14,  15,  16,  18,  21,  22,  23,  27,  28,  30,  50,  53,  54,
        56,  57,  59,  60,  61,  63,  64,  67,  68,  71,  94,  97,  98,
       102, 103, 104, 105, 106, 107, 113])